In [1]:
#Initial configuration, probably overkill in imports.
import sys, os, re
import numpy as np
import allel
import zarr
import dask
import numcodecs
import warnings
from pathlib import Path

#os.environ["MODIN_ENGINE"] = "ray"

#import modin.pandas as pd
import pandas as pd

%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina', 'png')
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
sns.set()
sns.set_theme()
sns.set_style("white")
sns.set_context("notebook")

In [2]:
#Dask setup
from multiprocessing.pool import ThreadPool
import dask
dask.config.set(pool=ThreadPool(int(os.environ['SLURM_CPUS_PER_TASK'])))

In [3]:
#Opening the zarr data
callset = zarr.open_group('/faststorage/project/primatediversity/people/kmt/baboon_flagship/steps/callset.zarr', mode='r')
callset.tree(expand=False)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [25]:
#Callability data
called_interv = zarr.open_group('/faststorage/project/primatediversity/people/kmt/baboon_flagship/steps/called_interv.zarr', mode='r')
#called_interv.tree(expand=False)

In [4]:
#Sample information and chromosome lengths
chromosome_lengths = dict()
for line in open('../data/macFas5.chrom.sizes.txt'): # NB MACAQUE REF GENOME...
    chrom, length = line.split()
    chromosome_lengths[chrom] = int(length)
    
chromosomes = [f'chr{x}' for x in range(1, 21)] + ['chrX']
meta_data = pd.read_excel('../data/Papio-Genomes_JR_120720_MR-CR-KM_geoloc.xlsx')
with pd.option_context("display.max_rows", 1000):
    display(meta_data)
baboon_samples = [x for x in meta_data.PGDP_ID if x.startswith('PD')] #  NB: to not get the SciAdvPaper samples
#Meta data for the sample present in the zarr data structure - Kasper has removed some of the samples.
samples_list = list(callset['chr1/samples'][:])
meta_data_samples = meta_data.loc[meta_data.PGDP_ID.isin(samples_list)].copy()
samples_callset_index = [samples_list.index(s) for s in meta_data_samples.PGDP_ID]
meta_data_samples['callset_index'] = samples_callset_index

,Unnamed: 0,PGDP_ID,Provider_ID,Provider,Genus,Species,Origin,Sex,address,longitude,latitude
0,0,PD_0067,1043,Roos,Theropithecus,gelada,captive,M,"SDSU Captive Wildlife Research Facility, Brook...",-96.793280,44.334031
1,1,PD_0199,09SNF1101115,Knauf/Chuma/Roos,Papio,anubis,"Serengeti, Tanzania",F,"Serengeti, Mara, Lake Zone, Tanzania",34.742544,-1.996626
2,2,PD_0200,11SNF1101115,Knauf/Chuma/Roos,Papio,anubis,"Serengeti, Tanzania",F,"Serengeti, Mara, Lake Zone, Tanzania",34.742544,-1.996626
3,3,PD_0201,19SNM1131115,Knauf/Chuma/Roos,Papio,anubis,"Serengeti, Tanzania",M,"Serengeti, Mara, Lake Zone, Tanzania",34.742544,-1.996626
4,4,PD_0202,20SNF1131115,Knauf/Chuma/Roos,Papio,anubis,"Serengeti, Tanzania",F,"Serengeti, Mara, Lake Zone, Tanzania",34.742544,-1.996626
5,5,PD_0203,21SNF1151115,Knauf/Chuma/Roos,Papio,anubis,"Serengeti, Tanzania",F,"Serengeti, Mara, Lake Zone, Tanzania",34.742544,-1.996626
6,6,PD_0204,03ANM1080815,Knauf/Chuma/Roos,Papio,anubis,"Arusha, Tanzania",M,"Arusha, Northern Zone, Tanzania",36.708585,-3.427534
7,7,PD_0205,05ANF1030116,Knauf/Chuma/Roos,Papio,anubis,"Arusha, Tanzania",F,"Arusha, Northern Zone, Tanzania",36.708585,-3.427534
8,8,PD_0206,09ANF1040116,Knauf/Chuma/Roos,Papio,anubis,"Arusha, Tanzania",F,"Arusha, Northern Zone, Tanzania",36.708585,-3.427534
9,9,PD_0207,12ANM1050116,Knauf/Chuma/Roos,Papio,anubis,"Arusha, Tanzania",M,"Arusha, Northern Zone, Tanzania",36.708585,-3.427534


In [6]:
meta_data_samples.head()

,Unnamed: 0,PGDP_ID,Provider_ID,Provider,Genus,Species,Origin,Sex,address,longitude,latitude,callset_index
1,1,PD_0199,09SNF1101115,Knauf/Chuma/Roos,Papio,anubis,"Serengeti, Tanzania",F,"Serengeti, Mara, Lake Zone, Tanzania",34.742544,-1.996626,0
2,2,PD_0200,11SNF1101115,Knauf/Chuma/Roos,Papio,anubis,"Serengeti, Tanzania",F,"Serengeti, Mara, Lake Zone, Tanzania",34.742544,-1.996626,1
3,3,PD_0201,19SNM1131115,Knauf/Chuma/Roos,Papio,anubis,"Serengeti, Tanzania",M,"Serengeti, Mara, Lake Zone, Tanzania",34.742544,-1.996626,2
4,4,PD_0202,20SNF1131115,Knauf/Chuma/Roos,Papio,anubis,"Serengeti, Tanzania",F,"Serengeti, Mara, Lake Zone, Tanzania",34.742544,-1.996626,3
5,5,PD_0203,21SNF1151115,Knauf/Chuma/Roos,Papio,anubis,"Serengeti, Tanzania",F,"Serengeti, Mara, Lake Zone, Tanzania",34.742544,-1.996626,4


In [7]:
#Picking out anubis based on metadata
anubis = meta_data_samples[meta_data_samples.Species == "anubis"].callset_index.values
anubis
#Example on how to load data for anubis on chrom 20
gt_zarr = callset["chr20/calldata/GT"]
gt = allel.GenotypeDaskArray(gt_zarr)
gt_anubis = gt.take(anubis, axis=1)
gt_anubis

<GenotypeDaskArray shape=(3328367, 51, 2) dtype=int8>

In [5]:
#Test subset of individuals
test_subset = meta_data_samples.take([0, 10, 144,])
test_subset

,Unnamed: 0,PGDP_ID,Provider_ID,Provider,Genus,Species,Origin,Sex,address,longitude,latitude,callset_index
1,1,PD_0199,09SNF1101115,Knauf/Chuma/Roos,Papio,anubis,"Serengeti, Tanzania",F,"Serengeti, Mara, Lake Zone, Tanzania",34.742544,-1.996626,0
11,11,PD_0209,05NCF1150516,Knauf/Chuma/Roos,Papio,anubis,"Ngorongoro, Tanzania",F,"Ngorongoro, Arusha, Northern Zone, Tanzania",35.585946,-2.676953,10
201,201,PD_0778,34390_BZ11034,Rogers/Jolly/Phillips-Conroy,Papio,kindae,"Chunga, Zambia",M,"Chunga, Mumbwa District, Central Province, Zambia",26.005210,-15.053557,144


In [6]:
#Loading chromosome lengths for the reference.
chrom_length_all = pd.read_table("../data/macFas5.chrom.sizes.txt", names = ["chrom", "size"])
chrom_length = chrom_length_all[:21]
print(chrom_length)
#Updating it so it is ordered based on based on standard ordering
ordered_sizes = []
for chrom in chromosomes:
    size = chrom_length.loc[chrom_length.chrom==chrom, "size"].iloc[0]
    ordered_sizes.append(size)
ordered_chrom_length = pd.DataFrame(list(zip(chromosomes, ordered_sizes)), columns = ["chrom", "size"])
ordered_chrom_length

    chrom       size
0    chr1  227556264
1    chr2  192460366
2    chr3  192294377
3    chr5  189454096
4    chr6  181584905
5    chr7  171882078
6    chr4  170955103
7    chrX  152835861
8    chr8  146850525
9   chr11  137757926
10   chr9  133195287
11  chr12  132586672
12  chr14  130733371
13  chr15  112612857
14  chr13  111193037
15  chr17   96864807
16  chr10   96509753
17  chr16   80997621
18  chr20   78541002
19  chr18   75711847
20  chr19   59248254


,chrom,size
0,chr1,227556264
1,chr2,192460366
2,chr3,192294377
3,chr4,170955103
4,chr5,189454096
5,chr6,181584905
6,chr7,171882078
7,chr8,146850525
8,chr9,133195287
9,chr10,96509753


In [9]:
#Setting up the dataframe, columns with placing information.
#Might just do it during the intervals.
window_size = 100000
chr_list = []
window_numbering = []
for i, row in ordered_chrom_length.iterrows():
    window_number = row["size"]//window_size #Add 1 if it is not ==
    chr_list.extend([row["chrom"] for i in range(window_number)])
    window_numbering.extend(range(window_number))
het_windows = pd.DataFrame(list(zip(chr_list, window_numbering)), columns = ["chrom", "window"])

In [10]:
#Checking that it is set up correct
#print(len(chr_list), len(window_numbering))
#het_windows.loc[het_windows["chrom"] == "chr1"]

In [28]:
#Check of length of callability versus POS
chrom = "chrX"
ID = "PD_0201"
pos = callset["{}/variants/POS".format(chrom)]
call_m = call_masks["{}/{}".format(chrom, ID)][:]
print("last variable site in zarr", pos[-1])
print("Length of callmask", len(call_m)) #there is a discrepancy between the call mask variable positions.
#Also checking the vcf
vcf_dir = "/faststorage/project/primatediversity/data/variants/"
vcf_suffix = ".variable.filtered.HF.snps.vcf.gz"
vcf_callset = allel.read_vcf(vcf_dir+ID+vcf_suffix, region="chrX", fields=['variants/POS', 'calldata/GT',])
print("last variable site in vcf for PD_0201", vcf_callset['variants/POS'][-1])
#Checking the called_intervs
print("last callability interval", called_interv["{}/{}".format(ID, chrom)][-1])

last variable site in zarr 153366679
Length of callmask 152835861
last variable site in vcf for PD_0201 153315293
last callability interval [153368040 153368048]


In [23]:
#Also checking the callability (loaning code from Kasper)
import gzip
call_dir = "/faststorage/project/primatediversity/data/callability/"
call_suffix = ".variable.filtered.callable.bed.gz"
bed_file = call_dir+ID+call_suffix
chrX_intervs = []
with gzip.open(bed_file, 'rt') as f:
    for line in f:
        chrom, start, end = line.split()
        # Only looking at X
        if chrom.startswith('chrX'):
            chrX_intervs.append(int(end)) 

In [24]:
max(chrX_intervs)

153368048

In [33]:
#Iterating over the individuals 
#Window size and chromosomes, as defined earlier, is used.
window_size = 100000
def het_counting(gt):
    return gt.count_het()


for chrom in ["chrX"]:
    gt_zarr = callset["{}/calldata/GT".format(chrom)]
    pos = callset["{}/variants/POS".format(chrom)]
    gt = allel.GenotypeArray(gt_zarr)
    df_list = []
    for i, row in meta_data_samples.iterrows():
        df = pd.DataFrame()
        individual = (gt.take([row.callset_index], axis=1))
        nnz, windows, counts = allel.windowed_statistic(pos, individual, statistic=het_counting, size=window_size)
        df["het"] = nnz
        if i % 10 == 0:
            print(i)
        window_numbering = []
        df.insert(0, column="chr", value=chrom)
        window_numbering.extend(range(len(nnz)))
        df.insert(1, column="window", value=window_numbering)
        df.insert(2, column = "PGDP_ID", value = row.PGDP_ID)
        df_list.append(df)
    chr_df = pd.concat(df_list, axis=0)
    chr_df.to_csv("../steps/het_counts_windows_{}.txt".format(chrom), sep = " ", index=False)
    print("Finished with {}".format(chrom))

10
20
30
40
60
120
130
140
150
160
170
180
190
200
210
Finished with chrX


In [11]:
#full_df.to_csv("../steps/het_counts_windows.txt", sep = " ")
full_df = pd.concat(df_list, axis=0)
full_df

,chr,window,PGDP_ID,het
0,chrX,0,PD_0199,128.0
1,chrX,1,PD_0199,75.0
2,chrX,2,PD_0199,NaN
3,chrX,3,PD_0199,121.0
4,chrX,4,PD_0199,158.0
...,...,...,...,...
1529,chrX,1529,PD_0793,1086.0
1530,chrX,1530,PD_0793,512.0
1531,chrX,1531,PD_0793,759.0
1532,chrX,1532,PD_0793,1334.0


In [14]:
full_df.loc[full_df.chr=="chrX"]

,chr,window,PGDP_ID,het
0,chrX,0,PD_0199,128.0
1,chrX,1,PD_0199,75.0
2,chrX,2,PD_0199,NaN
3,chrX,3,PD_0199,121.0
4,chrX,4,PD_0199,158.0
...,...,...,...,...
1529,chrX,1529,PD_0793,1086.0
1530,chrX,1530,PD_0793,512.0
1531,chrX,1531,PD_0793,759.0
1532,chrX,1532,PD_0793,1334.0


In [10]:
#Using the new callability Kasper generated
call_masks = zarr.open_group('/faststorage/project/primatediversity/people/kmt/baboon_flagship/steps/call_masks.zarr', mode='r')

In [31]:
#The for loop. Calculating percentage coverage for every window
window_size = 100000
for chrom in chromosomes:
    df_list = []
    for i, row in meta_data_samples.iterrows():
        df = pd.DataFrame()
        individual = call_masks["{}/{}".format(chrom, row.PGDP_ID)][:]
        size = len(individual)
        coverage = []
        window_end = 0
        while window_end < size-window_size:
            coverage.append(individual[window_end:window_end+window_size].sum()/window_size)
            window_end += window_size
        coverage.append(individual[window_end:].sum()/(size-window_end))
        df["coverage"] = coverage
        if i % 10 == 0:
            print(i)
        window_numbering = []
        df.insert(0, column="chr", value=chrom)
        window_numbering.extend(range(len(coverage)))
        df.insert(1, column="window", value=window_numbering)
        df.insert(2, column = "PGDP_ID", value = row.PGDP_ID)
        df_list.append(df)
    chr_df = pd.concat(df_list, axis=0)
    chr_df.to_csv("../steps/callability_windows_{}.txt".format(chrom), sep = " ", index=False)
    print("Finished with {}".format(chrom))

10
20
30
40
60
120
130
140
150
160
170
180
190
200
210
Finished with chr1
10
20
30
40
60
120
130
140
150
160
170
180
190
200
210
Finished with chr2
10
20
30
40
60
120
130
140
150
160
170
180
190
200
210
Finished with chr3
10
20
30
40
60
120
130
140
150
160
170
180
190
200
210
Finished with chr4
10
20
30
40
60
120
130
140
150
160
170
180
190
200
210
Finished with chr5
10
20
30
40
60
120
130
140
150
160
170
180
190
200
210
Finished with chr6
10
20
30
40
60
120
130
140
150
160
170
180
190
200
210
Finished with chr7
10
20
30
40
60
120
130
140
150
160
170
180
190
200
210
Finished with chr8
10
20
30
40
60
120
130
140
150
160
170
180
190
200
210
Finished with chr9
10
20
30
40
60
120
130
140
150
160
170
180
190
200
210
Finished with chr10
10
20
30
40
60
120
130
140
150
160
170
180
190
200
210
Finished with chr11
10
20
30
40
60
120
130
140
150
160
170
180
190
200
210
Finished with chr12
10
20
30
40
60
120
130
140
150
160
170
180
190
200
210
Finished with chr13
10
20
30
40
60
120
130
140
150
160

In [32]:
chr_df["coverage"].mean()

0.8419892866554879

#Calculating length of intervals
##Redoing it with a new implementation
df_call_list = []
c = 0
for chrom in chromosomes:
    df = pd.DataFrame()
    pos = callset["{}/variants/POS".format(chrom)]
    chrom_size = pos[-1]
    for index, row in test_subset.iterrows():
        path = "{}/{}".format(row.PGDP_ID, chrom)
        i = called_interv[path][:]
        s = 0
        window_end = window_size
        window_list = []
        for interval in i:
            while interval[0] >= window_end:
                window_list.append(s)
                window_end += window_size
                s = interval[1]-interval[0]
            elif interval[1] >= window_end:
                s += window_end-interval[0]
                window_list.append(s)
                s = interval[1]-window_end
                window_end += window_size
                if interval[1] > window_end:
                    c += 1
            else:
                s += interval[1]-interval[0]
        window_list.append(s)
        while int(chrom_size) > window_end:
            window_list.append(0)
            window_end += window_size
        if index % 10 == 0:
            print(index)
        df[row.PGDP_ID] = window_list
    df.insert(0, column="chr", value=chrom)
    df_call_list.append(df)
    print("Finished {}".format(chrom))
full_call_df = pd.concat(df_call_list, axis=0, ignore_index=True)
full_call_df.to_csv("../steps/call_counts_windows.txt", sep = " ")

In [15]:
full_call_df

,chr,PD_0199,PD_0209,PD_0778
0,chr1,80327,77962,80350
1,chr1,91796,91748,91221
2,chr1,70480,70085,70512
3,chr1,32129,32072,38519
4,chr1,58661,57475,57529
...,...,...,...,...
28427,chrX,87806,87034,90464
28428,chrX,92136,91382,92146
28429,chrX,78585,78049,78691
28430,chrX,88794,86936,87041


In [16]:
c

747

In [ ]:
full_call_df

In [ ]:
full_call_df.loc[full_call_df.chr=="chrX"]

In [ ]:
#General purpose plots
chrX_df = full_df.loc[full_df.chr=="chrX"]

In [ ]:
#Investigating heterozygosity in males on the X